# Procesamiento Licitaciones BO

## 📌 Cargar Datos desde Bronze


In [1]:
import pandas as pd
import numpy
from deltalake import DeltaTable, write_deltalake

from scripts.transformaciones_licitaciones import (
    extraer_campos_licitacion, 
    limpiar_texto_licitacion,
    extraer_monto_total,
    procesar_licitaciones_con_progreso
)
from scripts.transformaciones_empresas import generar_df_empresas_ocid
from scripts.transformaciones_licitaciones import (
    extraer_campos_licitacion,
    limpiar_texto_licitacion,
    extraer_monto_total,
)

from collections import defaultdict
from scripts.storage import upsert_data
from scripts.transformaciones_licitaciones import procesar_licitaciones_con_progreso


In [2]:
# ─────────────────────────────
# Función  para deduplicar por PK
# ─────────────────────────────
def deduplicar_por_pk(df, pk_col, fecha_col="fecha_extraccion"):
    if fecha_col in df.columns:
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
        df = df.sort_values(by=fecha_col, ascending=False)
    return df.drop_duplicates(subset=pk_col, keep='first').reset_index(drop=True)

In [3]:
# ─────────────────────────────
#  rutas de origen (Bronze)
# ─────────────────────────────
BRONZE_PATH = "data/bronze/boletin_api"
SILVER_PATH = "data/silver/boletin_api"
# Rutas a las tablas Bronze
BRONZE_BOLETINES = f"{BRONZE_PATH}/boletines"
BRONZE_NORMAS = f"{BRONZE_PATH}/normas"
BRONZE_LICITACIONES = f"{BRONZE_PATH}/licitaciones"
BRONZE_ORGANISMOS = f"{BRONZE_PATH}/organismos_emisores"
BRONZE_REPARTICIONES = f"{BRONZE_PATH}/reparticiones"
BRONZE_EMPRESAS = "data/bronze/bac_anual/full_bac_compras_anual"


# ─────────────────────────────
# Carga de tablas desde Delta Lake
# ─────────────────────────────
dt_boletines = DeltaTable(BRONZE_BOLETINES)
dt_normas = DeltaTable(BRONZE_NORMAS)
dt_licitaciones = DeltaTable(BRONZE_LICITACIONES)
dt_organismos = DeltaTable(BRONZE_ORGANISMOS)
dt_reparticiones = DeltaTable(BRONZE_REPARTICIONES)
dt_ocid = DeltaTable(BRONZE_EMPRESAS)

In [4]:


df_boletines = deduplicar_por_pk(dt_boletines.to_pandas(), pk_col="numero")
df_normas = deduplicar_por_pk(dt_normas.to_pandas(), pk_col="id_norma")
df_licitaciones = deduplicar_por_pk(dt_licitaciones.to_pandas(), pk_col="id_norma")
df_organismos = dt_organismos.to_pandas()
df_reparticiones = dt_reparticiones.to_pandas()
df_ocid = dt_ocid.to_pandas()

In [5]:

# ─────────────────────────────
# Mostrar resultados y estructura
# ─────────────────────────────
print("📌 Boletines (deduplicados):")
display(df_boletines.head())
print("📌 Normas (deduplicadas):")
display(df_normas.head())
print("📌 Licitaciones (deduplicadas):")
display(df_licitaciones.head())
print("📌 Organismos (deduplicados):")
display(df_organismos.head())
print("📌 Reparticiones (deduplicadas):")
display(df_reparticiones.head())
print("📌 Empresas (desde Bronze - OCID):")
display(df_ocid.head())

print("\n🔍 Info OCID:")
df_ocid.info()
print("\n🔍 Info Boletines:")
df_boletines.info()
print("\n🔍 Info Normas:")
df_normas.info()
print("\n🔍 Info Licitaciones:")
df_licitaciones.info()
print("\n🔍 Info Organismos Emisores:")
df_organismos.info()
print("\n🔍 Info Reparticiones:")
df_reparticiones.info()


📌 Boletines (deduplicados):


,fecha_publicacion,mes,dia,anio,numero,numero2,nombre,url_boletin,separata
0,2025-04-11,04,11,2025,7098,None,None,http://api-restboletinoficial.buenosaires.gob....,"[{'nombre': None, 'url': 'http://api-restbolet..."


📌 Normas (deduplicadas):


,fecha_publicacion,subsecciones,tipo_norma,organismo,nombre,sumario,id_norma,url_norma,anexos,id_sdin
0,11/04/2025,Comunicados y Avisos,Ley 6009 - Regulación identificación de los ap...,Ministerio de Justicia,Ley 6009 - Regulación identificación de los ap...,Actas de secuestro en el marco del operativo r...,883370,http://api-restboletinoficial.buenosaires.gob....,[{'filenet_firmado': 'http://api-restboletinof...,
1,11/04/2025,Comunicados y Avisos,Ley 6009 - Regulación identificación de los ap...,Ministerio de Justicia,Ley 6009 - Regulación identificación de los ap...,Actas de secuestro en el marco del operativo r...,883365,http://api-restboletinoficial.buenosaires.gob....,[{'filenet_firmado': 'http://api-restboletinof...,
2,11/04/2025,Comunicados y Avisos,Ley N° 1356 de Calidad Atmosférica,Vicejefatura de Gobierno,Ley N° 1356 de Calidad Atmosférica N° 1/DGCONT...,Informes de Monitoreo Atmosférico Automático d...,884254,http://api-restboletinoficial.buenosaires.gob....,[{'filenet_firmado': 'http://api-restboletinof...,
3,11/04/2025,Comunicados y Avisos,Ley N° 1356 de Calidad Atmosférica,Vicejefatura de Gobierno,Ley N° 1356 de Calidad Atmosférica N° 2/DGCONT...,Informes de Monitoreo Atmosférico Automático d...,884257,http://api-restboletinoficial.buenosaires.gob....,[{'filenet_firmado': 'http://api-restboletinof...,
4,11/04/2025,Comunicados y Avisos,Ley N° 1356 de Calidad Atmosférica,Vicejefatura de Gobierno,Ley N° 1356 de Calidad Atmosférica N° 3/DGCONT...,Informes de Monitoreo Atmosférico Automático d...,884266,http://api-restboletinoficial.buenosaires.gob....,[{'filenet_firmado': 'http://api-restboletinof...,


📌 Licitaciones (deduplicadas):


...

📌 Reparticiones (deduplicadas):


,reparticion_id,nombre,sigla,fecha_extraccion
0,13,Administración Gubernamental de Ingresos Públicos,AGIP,2025-04-13
1,16,Agencia de Protección Ambiental,APRA,2025-04-13
2,20,Agencia de Sistemas de Información,ASINF,2025-04-13
3,14,Agencia Gubernamental de Control,AGC,2025-04-13
4,23,Área Jefe de Gobierno,AJG,2025-04-13


📌 Empresas (desde Bronze - OCID):


,ocid,id,date,initiationType,tag,tender/id,tender/title,tender/description,tender/status,tender/procuringEntity/id,...,parties/0/address/countryName,parties/0/contactPoint/name,parties/0/contactPoint/email,parties/0/contactPoint/telephone,parties/0/contactPoint/url,relatedProcesses/id,relatedProcesses/relationship,relatedProcesses/title,relatedProcesses/scheme,relatedProcesses/identifier
0,ocds-bulbcf-10002-0078-LPU25-1-0,10002-0078-LPU25-1-0,2025-02-11T11:00:00-03:00,tender,tender,10002-0078-LPU25,Licitación Pública Espacio en el interior de l...,Otros no especificados precedentemente-,active,CABA-UE-10002,...,Republica Argentina,Mesa de Ayuda BAC,mesadeayudabac@dguiaf-gcba.gob.ar,5.401152e+12,https://www.buenosairescompras.gob.ar,NaN,None,None,None,NaN
1,ocds-bulbcf-10002-1573-LPU24-1-0,10002-1573-LPU24-1-0,2025-03-13T11:00:00-03:00,tender,tender,10002-1573-LPU24,Bajo Autopista 25 de Mayo -Ley N° 6.056-Tte. M...,Otros no especificados precedentemente-,active,CABA-UE-10002,...,Republica Argentina,Mesa de Ayuda BAC,mesadeayudabac@dguiaf-gcba.gob.ar,5.401152e+12,https://www.buenosairescompras.gob.ar,NaN,None,None,None,NaN
2,ocds-bulbcf-101-0240-CME25-1-0,101-0240-CME25-1-0,2025-02-13T12:00:00-03:00,tender,tender;award;contract,101-0240-CME25,Adquisición de una Cámara Digital con Lente y ...,"Equipo educacional, cultural y recreativo-",complete,CABA-UE-101,...,Republica Argentina,Mesa de Ayuda BAC,mesadeayudabac@dguiaf-gcba.gob.ar,5.401152e+12,https://www.buenosairescompras.gob.ar,NaN,None,None,None,NaN
3,ocds-bulbcf-101-0240-CME25-1-1,101-0240-CME25-1-1,2025-02-13T12:00:00-03:00,tender,tender;award;contract,101-0240-CME25,Adquisición de una Cámara Digital con Lente y ...,"Equipo educacional, cultural y recreativo-",complete,CABA-UE-101,...,None,None,None,NaN,None,NaN,None,None,None,NaN
4,ocds-bulbcf-101-0240-CME25-2-0,101-0240-CME25-2-0,2025-02-13T12:00:00-03:00,tender,tender;award;contract,101-0240-CME25,Adquisición de una Cámara Digital con Lente y ...,"Equipo educacional, cultural y recreativo-",complete,CABA-UE-101,...,None,None,None,NaN,None,NaN,None,None,None,NaN



🔍 Info OCID:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10291 entries, 0 to 10290
Columns: 113 entries, ocid to relatedProcesses/identifier
dtypes: bool(2), float64(19), int64(2), object(90)
memory usage: 8.7+ MB

🔍 Info Boletines:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha_publicacion  1 non-null      datetime64[us]
 1   mes                1 non-null      object        
 2   dia                1 non-null      object        
 3   anio               1 non-null      int32         
 4   numero             1 non-null      int64         
 5   numero2            1 non-null      object        
 6   nombre             1 non-null      object        
 7   url_boletin        1 non-null      object        
 8   separata           1 non-null      object        
dtypes: datetime64[us](1), int32(1), int64(1), object(6

# Transformaciones

## Empresas

Filtro y selección de empresas en base a información de compras en formato OCID obtenidas del padrón de Datos Abiertos de CABA

In [6]:

# ─────────────────────────────
# Generar listado de empresas ofertantes en base a BAC 
# ─────────────────────────────

df_empresas = generar_df_empresas_ocid(df_ocid)

print(df_empresas.head())

                       company_id                    company_name  \
0  AR-CUIT-20-23881274-8-supplier  Facundo Adrian Ortiz Guerreiro   
1  AR-CUIT-30-71711268-3-supplier                      Eficci srl   
2  AR-CUIT-30-70788862-4-supplier              ST PRODUCCIONES SA   
3    AR-CUIT-30650273652-supplier            VIVERO CUCULO S.R.L.   
4  AR-CUIT-20-23201143-3-supplier           Waldo QUIROGA NICOLÁS   

  pais_empresa   cuit_empresa         company_name_normalized  
0           AR  20-23881274-8  FACUNDO ADRIAN ORTIZ GUERREIRO  
1           AR  30-71711268-3                   EFICCI S.R.L.  
2           AR  30-70788862-4            ST PRODUCCIONES S.A.  
3           AR  30-65027365-2            VIVERO CUCULO S.R.L.  
4           AR  20-23201143-3           WALDO QUIROGA NICOLÁS  


# Procesamiento de Licitaciones

Limpieza de texto, identificación de etapas de licitaciones

In [7]:
# ─────────────────────────────
# 1️⃣ PARSEO Y LIMPIEZA DE LICITACIONES
# ─────────────────────────────

# 📌 Extraer tipo, etapa y código de licitación
df_extraccion = df_licitaciones.apply(extraer_campos_licitacion, axis=1, result_type='expand')
df_licitaciones[["tipo_licitacion", "etapa_licitacion", "codigo_licitacion"]] = df_extraccion

print("\n📊 Valores únicos de tipo_licitacion:")
print(df_licitaciones["tipo_licitacion"].value_counts())

# 🧼 Limpieza del texto de licitación
df_licitaciones["texto_limpio"] = df_licitaciones["texto_licitaciones"].apply(limpiar_texto_licitacion)



📊 Valores únicos de tipo_licitacion:
tipo_licitacion
Licitación Pública         21
Licitación Obra Pública    10
Contratación Menor          9
Contratación Directa        1
Name: count, dtype: int64


# Identificación de Montos

Cálculo de monto total adjudicado por licitación

In [8]:
# ─────────────────────────────
# MONTOS Y CUITs
# ─────────────────────────────

# extracción de monto en etapas específicas
etapas_validas = ['preadjudicacion', 'adjudicacion', 'prorroga']

df_licitaciones['monto_total'] = df_licitaciones.apply(
    lambda row: extraer_monto_total(row['texto_limpio'])
    if isinstance(row['texto_limpio'], str) and str(row['etapa_licitacion']).lower() in etapas_validas else 0,
    axis=1
)

## Identificación de empresas en etapas seleccionadas (Fuzzy Match)

In [9]:
# ─────────────────────────────
#  IDENTIFICACIÓN DE EMPRESAS (Fuzzy Match)
# ─────────────────────────────


# 👇 etapas relevantes
etapas_relevantes = ['prorroga', 'adjudicacion', 'preadjudicacion']
mask = df_licitaciones['etapa_licitacion'].str.lower().str.strip().isin(etapas_relevantes)
licitaciones_filtradas = df_licitaciones[mask].copy()

# 🔍 Ejecutar fuzzy matching
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

print(f"🚀 Iniciando fuzzy matching de {len(licitaciones_filtradas)} licitaciones...")

licitaciones_filtradas["empresas_identificadas"] = procesar_licitaciones_con_progreso(
    licitaciones_filtradas,
    df_empresas
)

print("✅ ¡Fuzzy Matching completado!")


🚀 Iniciando fuzzy matching de 15 licitaciones...


  0%|          | 0/15 [00:00<?, ?it/s]2025-04-13 12:20:55,154 - ID: 883910 | Empresa: BIODIAGNOSTICO S.A. (CUIT: 30-63927711-5) | Score: 100
2025-04-13 12:20:55,169 - ID: 884084 | Empresa: LABORATORIOS IGALTEX S.R.L. (CUIT: 30-61680618-8) | Score: 100
2025-04-13 12:20:55,182 - ID: 883984 | Empresa: TECNONUCLEAR S.A. (CUIT: 30-64250768-7) | Score: 100
2025-04-13 12:20:55,195 - ID: 884218 | Empresa: WM ARGENTINA S.A. (CUIT: 30-68617257-7) | Score: 90
2025-04-13 12:20:55,219 - ID: 884113 | Empresa: LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL Y COMERCIAL (CUIT: 30-57076089-7) | Score: 100
 40%|████      | 6/15 [00:00<00:00, 55.52it/s]2025-04-13 12:20:55,319 - ID: 884291 | Empresa: FAMOX S.A. (CUIT: 30-70849490-5) | Score: 100
2025-04-13 12:20:55,335 - ID: 884216 | Empresa: BIO-SAN ARGENTINA S.R.L. (CUIT: 30-71711355-8) | Score: 100
100%|██████████| 15/15 [00:00<00:00, 65.55it/s]

✅ ¡Fuzzy Matching completado!


Análais de consistencia

In [10]:
total_licitaciones = len(licitaciones_filtradas)
con_empresas = licitaciones_filtradas["empresas_identificadas"].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()
total_empresas_identificadas = licitaciones_filtradas["empresas_identificadas"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
).sum()

print(f"\n📊 Resumen de identificación de empresas:")
print(f"🔹 Licitaciones procesadas: {total_licitaciones}")
print(f"🔹 % de Licitaciones con al menos una empresa detectada: {round(con_empresas/ total_licitaciones, 2)*100}%" )
print(f"🔹 Total de presentaciones de empresas (suma de todas): {total_empresas_identificadas}")



📊 Resumen de identificación de empresas:
🔹 Licitaciones procesadas: 15
🔹 % de Licitaciones con al menos una empresa detectada: 47.0%
🔹 Total de presentaciones de empresas (suma de todas): 7


In [11]:
df_licitaciones = df_licitaciones.drop(columns=['empresas_identificadas'], errors='ignore')

df_licitaciones = df_licitaciones.merge(
    licitaciones_filtradas[["id_norma", "empresas_identificadas"]],
    on="id_norma",
    how="left"
)


In [12]:
df_licitaciones = df_licitaciones.drop(['__index_level_0__', 'texto_licitaciones'], axis = 1)

## Enriquecimiento df_empresas

Se agrega a cada empresa el recuento de presentaciones en actos administrativo de adjudicación, preadjudicación o prorroga de licitaciones públicas Se discrimina según empresas adjudicatarias.

In [13]:
# Inicializar contadores
presentaciones = defaultdict(int)
presentaciones_adjudicacion = defaultdict(int)

# Iterar por cada fila de licitaciones
for _, row in df_licitaciones.iterrows():
    empresas = row.get("empresas_identificadas", [])
    etapa = row.get("etapa_licitacion")

    etapa = etapa.lower().strip() if isinstance(etapa, str) else ""

    if isinstance(empresas, list):
        for empresa in empresas:
            cuit = empresa.get("cuit")
            if cuit:
                presentaciones[cuit] += 1
                if etapa == "adjudicacion":
                    presentaciones_adjudicacion[cuit] += 1

# Convertir a DataFrame
df_score = pd.DataFrame([
    {
        "cuit_empresa": cuit,
        "total_presentaciones": presentaciones[cuit],
        "presentaciones_adjudicacion": presentaciones_adjudicacion.get(cuit, 0)
    }
    for cuit in presentaciones
])

# Unir al DataFrame de empresas
df_empresas = df_empresas.merge(df_score, on="cuit_empresa", how="left")
df_empresas[["total_presentaciones", "presentaciones_adjudicacion"]] = df_empresas[["total_presentaciones", "presentaciones_adjudicacion"]].fillna(0).astype(int)

# Ejemplo
df_empresas[["cuit_empresa", "company_name_normalized", "total_presentaciones", "presentaciones_adjudicacion"]].head()

df_empresas = df_empresas.sort_values(["presentaciones_adjudicacion", "total_presentaciones"], ascending=[False, False])


In [14]:
# Top 10 empresas más adjudicadas
top_empresas = df_empresas.sort_values(["presentaciones_adjudicacion", "total_presentaciones"], ascending=[False, False]).head(30)
print("\n🏆 Top 30 empresas por adjudicaciones:")
top_empresas[["cuit_empresa", "company_name_normalized", "total_presentaciones", "presentaciones_adjudicacion"]]



🏆 Top 30 empresas por adjudicaciones:


,cuit_empresa,company_name_normalized,total_presentaciones,presentaciones_adjudicacion
206,30-71711355-8,BIO-SAN ARGENTINA S.R.L.,1,1
53,30-57076089-7,LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL...,1,0
123,30-63927711-5,BIODIAGNOSTICO S.A.,1,0
127,30-61680618-8,LABORATORIOS IGALTEX S.R.L.,1,0
148,30-64250768-7,TECNONUCLEAR S.A.,1,0
199,30-68617257-7,WM ARGENTINA S.A.,1,0
239,30-70849490-5,FAMOX S.A.,1,0
0,20-23881274-8,FACUNDO ADRIAN ORTIZ GUERREIRO,0,0
1,30-71711268-3,EFICCI S.R.L.,0,0
2,30-70788862-4,ST PRODUCCIONES S.A.,0,0


In [15]:
sum(df_empresas['total_presentaciones'])

7

# Enriquecimiento de normas y licitaciones (join)

In [16]:
# Asegurarnos de que ambas fechas sean tipo datetime
df_normas["fecha_publicacion"] = pd.to_datetime(df_normas["fecha_publicacion"])
df_boletines["fecha_publicacion"] = pd.to_datetime(df_boletines["fecha_publicacion"])

# Hacemos el merge por fecha
df_normas = df_normas.merge(
    df_boletines[["fecha_publicacion", "numero"]],
    left_on = 'fecha_publicacion',
    right_on="fecha_publicacion",
    how="left"
)

In [17]:
df_licitaciones = df_licitaciones.merge(
    df_normas[["id_norma", "numero", "organismo"]],
    on="id_norma",
    how="left"
)

In [18]:
# ────────────────────────────────────────────────
# RESUMEN POR ORGANISMO: Adjudicaciones y Empresas
# ────────────────────────────────────────────────

# Filtrar solo licitaciones en etapa de adjudicación
df_adjudicadas = df_licitaciones[
    df_licitaciones["etapa_licitacion"].str.lower() == "adjudicacion"
].copy()

# Imputación de monto_total como numérico
df_adjudicadas["monto_total"] = pd.to_numeric(df_adjudicadas["monto_total"], errors="coerce").fillna(0)

In [19]:
# Expandir empresas
filas_expand = []

for _, row in df_adjudicadas.iterrows():
    organismo = row.get("organismo")
    monto = row.get("monto_total", 0)
    empresas = row.get("empresas_identificadas", [])
    
    if isinstance(empresas, list):
        for empresa in empresas:
            filas_expand.append({
                "organismo": organismo,
                "cuit_empresa": empresa.get("cuit"),
                "nombre_empresa": empresa.get("nombre"),
                "monto_total": monto
            })
df_empresas_adjudicadas = pd.DataFrame(filas_expand)


In [20]:
# Groubpy de monto total adjudicado y cantidad de empresas por organismo
resumen_organismo = (
    df_empresas_adjudicadas
    .groupby("organismo")
    .agg(
        monto_total_adjudicado=("monto_total", "sum"),
        cantidad_empresas=("cuit_empresa", "nunique")
    )
    .reset_index()
    .sort_values(by="monto_total_adjudicado", ascending=False)
)

resumen_organismo.head()

,organismo,monto_total_adjudicado,cantidad_empresas
0,Ministerio de Salud,1741664.18,1


In [21]:
# Función para obtener el organismo más frecuente
def organismo_top(diccionario):
    return max(diccionario.items(), key=lambda x: x[1])[0] if diccionario else None

In [22]:
# ────────────────────────────────────────────────
# ENRIQUECIMIENTO DE EMPRESAS CON ORGANISMOS
# ────────────────────────────────────────────────


# Inicializar contadores
organismos_adjudicacion = defaultdict(lambda: defaultdict(int))
organismos_presencia = defaultdict(lambda: defaultdict(int))

# Recorremos TODAS las licitaciones
for _, row in df_licitaciones.iterrows():
    organismo = row.get("organismo")
    etapa = row.get("etapa_licitacion", "").lower() if isinstance(row.get("etapa_licitacion"), str) else ""
    empresas = row.get("empresas_identificadas", [])

    if not isinstance(empresas, list):
        continue

    for empresa in empresas:
        cuit = empresa.get("cuit")
        if not cuit or not organismo:
            continue
        organismos_presencia[cuit][organismo] += 1
        if etapa == "adjudicacion":
            organismos_adjudicacion[cuit][organismo] += 1



#  DataFrame auxiliar
df_organismos_empresas = pd.DataFrame([
    {
        "cuit_empresa": cuit,
        "organismo_top_adjudicacion": organismo_top(organismos_adjudicacion[cuit]),
        "organismo_top_presencia": organismo_top(organismos_presencia[cuit]),
    }
    for cuit in organismos_presencia
])

# Merge al df_empresas
df_empresas = df_empresas.merge(df_organismos_empresas, on="cuit_empresa", how="left")

df_empresas[["organismo_top_adjudicacion", "organismo_top_presencia"]] = df_empresas[["organismo_top_adjudicacion", "organismo_top_presencia"]].fillna('<NA>')

# Mostrar preview
print("🏢 Empresas enriquecidas con organismos más frecuentes:")
display(df_empresas[["cuit_empresa", "company_name_normalized", "organismo_top_adjudicacion", "organismo_top_presencia"]].head())


🏢 Empresas enriquecidas con organismos más frecuentes:


,cuit_empresa,company_name_normalized,organismo_top_adjudicacion,organismo_top_presencia
0,30-71711355-8,BIO-SAN ARGENTINA S.R.L.,Ministerio de Salud,Ministerio de Salud
1,30-57076089-7,LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL...,<NA>,Ministerio de Salud
2,30-63927711-5,BIODIAGNOSTICO S.A.,<NA>,Ministerio de Salud
3,30-61680618-8,LABORATORIOS IGALTEX S.R.L.,<NA>,Ministerio de Salud
4,30-64250768-7,TECNONUCLEAR S.A.,<NA>,Ministerio de Salud


In [23]:
df_empresas

,company_id,company_name,pais_empresa,cuit_empresa,company_name_normalized,total_presentaciones,presentaciones_adjudicacion,organismo_top_adjudicacion,organismo_top_presencia
0,AR-CUIT-30-71711355-8-supplier,BIO-SAN ARGENTINA SRL,AR,30-71711355-8,BIO-SAN ARGENTINA S.R.L.,1,1,Ministerio de Salud,Ministerio de Salud
1,AR-CUIT-30-57076089-7-supplier,LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL...,AR,30-57076089-7,LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL...,1,0,<NA>,Ministerio de Salud
2,AR-CUIT-30-63927711-5-supplier,BIODIAGNOSTICO S.A.,AR,30-63927711-5,BIODIAGNOSTICO S.A.,1,0,<NA>,Ministerio de Salud
3,AR-CUIT-30-61680618-8-supplier,LABORATORIOS IGALTEX SRL,AR,30-61680618-8,LABORATORIOS IGALTEX S.R.L.,1,0,<NA>,Ministerio de Salud
4,AR-CUIT-30-64250768-7-supplier,TECNONUCLEAR S.A.,AR,30-64250768-7,TECNONUCLEAR S.A.,1,0,<NA>,Ministerio de Salud
...,...,...,...,...,...,...,...,...,...
313,AR-CUIT-30-71616658-5-supplier,MARCOS LUMEN SRL,AR,30-71616658-5,MARCOS LUMEN S.R.L.,0,0,<NA>,<NA>
314,AR-CUIT-30-62669896-0-supplier,DELMIRO MENDEZ E HIJO S.A.,AR,30-62669896-0,DELMIRO MENDEZ E HIJO S.A.,0,0,<NA>,<NA>
315,AR-CUIT-23-14867328-4-supplier,Adriana Monica Maestri,AR,23-14867328-4,ADRIANA MONICA MAESTRI,0,0,<NA>,<NA>
316,AR-CUIT-20-11897777-8-supplier,Carlos Bolig,AR,20-11897777-8,CARLOS BOLIG,0,0,<NA>,<NA>


In [24]:
resumen_organismo['organismo'].value_counts()

organismo
Ministerio de Salud    1
Name: count, dtype: int64

In [25]:
#Casteo de columna 'organismo'
resumen_organismo['organismo'] = resumen_organismo['organismo'].astype('category')


# Carga en Silver

In [26]:

# 📂 Definir rutas Silver
SILVER_PATH = "data/silver/boletin_api"
SILVER_BOLETINES = f"{SILVER_PATH}/boletines"
#SILVER_NORMAS = f"{SILVER_PATH}/normas"
SILVER_LICITACIONES = f"{SILVER_PATH}/licitaciones"
SILVER_ORGANISMOS = f"{SILVER_PATH}/organismos_emisores"
#SILVER_REPARTICIONES = f"{SILVER_PATH}/reparticiones"
SILVER_EMPRESAS = "data/silver/bac_anual/empresas"



In [27]:
# 💾 Guardar en Silver
write_deltalake(SILVER_BOLETINES, df_boletines, mode="overwrite")
#write_deltalake(SILVER_NORMAS, df_normas, mode="overwrite")
write_deltalake(SILVER_LICITACIONES, df_licitaciones, mode="overwrite")
write_deltalake(SILVER_ORGANISMOS, resumen_organismo, mode="overwrite")
#write_deltalake(SILVER_REPARTICIONES, df_reparticiones, mode="overwrite")
write_deltalake(SILVER_EMPRESAS, df_empresas, mode="overwrite")

print("\n✅ Todos los datos fueron guardados correctamente en Silver.")


✅ Todos los datos fueron guardados correctamente en Silver.


# Prueba Silver

In [28]:
# Ruta donde guardaste la tabla de empresas en Silver
SILVER_EMPRESAS = "data/silver/bac_anual/empresas"

# Cargar la tabla Delta como DataFrame
dt_empresas = DeltaTable(SILVER_EMPRESAS)
df_empresas_guardadas = dt_empresas.to_pandas()

# Mostrar las primeras filas
print("📦 Empresas guardadas en Silver:")
display(df_empresas_guardadas.head())

# Info general para verificar tipos y columnas
print("\n🔍 Información general de df_empresas_guardadas:")
df_empresas_guardadas.info()


📦 Empresas guardadas en Silver:


,company_id,company_name,pais_empresa,cuit_empresa,company_name_normalized,total_presentaciones,presentaciones_adjudicacion,organismo_top_adjudicacion,organismo_top_presencia
0,AR-CUIT-30-71711355-8-supplier,BIO-SAN ARGENTINA SRL,AR,30-71711355-8,BIO-SAN ARGENTINA S.R.L.,1,1,Ministerio de Salud,Ministerio de Salud
1,AR-CUIT-30-57076089-7-supplier,LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL...,AR,30-57076089-7,LABORATORIOS BACON SOCIEDAD ANONIMA INDUSTRIAL...,1,0,<NA>,Ministerio de Salud
2,AR-CUIT-30-63927711-5-supplier,BIODIAGNOSTICO S.A.,AR,30-63927711-5,BIODIAGNOSTICO S.A.,1,0,<NA>,Ministerio de Salud
3,AR-CUIT-30-61680618-8-supplier,LABORATORIOS IGALTEX SRL,AR,30-61680618-8,LABORATORIOS IGALTEX S.R.L.,1,0,<NA>,Ministerio de Salud
4,AR-CUIT-30-64250768-7-supplier,TECNONUCLEAR S.A.,AR,30-64250768-7,TECNONUCLEAR S.A.,1,0,<NA>,Ministerio de Salud



🔍 Información general de df_empresas_guardadas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   company_id                   318 non-null    object
 1   company_name                 318 non-null    object
 2   pais_empresa                 318 non-null    object
 3   cuit_empresa                 318 non-null    object
 4   company_name_normalized      318 non-null    object
 5   total_presentaciones         318 non-null    int64 
 6   presentaciones_adjudicacion  318 non-null    int64 
 7   organismo_top_adjudicacion   318 non-null    object
 8   organismo_top_presencia      318 non-null    object
dtypes: int64(2), object(7)
memory usage: 22.5+ KB
